# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [42]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [43]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [44]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [45]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [46]:
# This make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [47]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print("create key space error")
    print(e)

#### Set Keyspace

In [48]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print("set keyspace error")
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [49]:
## Create Table For Query 1:
## Get the artist, song title and song's length 
## For given sessionId and itemInSession 
query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(session_id int, item_in_session int, artist text, song text, song_length decimal, primary key (session_id, item_in_session))"
try:
    session.execute("DROP TABLE IF EXISTS sessions")
    session.execute(query)
except Exception as e:
    print(e)

## Create Table for Query 2:
## Get name of artist, song (sorted by itemInSession) and user (first and last name)\
## For given userid, sessionid
query = "CREATE TABLE IF NOT EXISTS users "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, primary key ((user_id, session_id), item_in_session))"
try:
    session.execute("DROP TABLE IF EXISTS users")
    session.execute(query)
except Exception as e:
    print(e)
                
## Create Table for Query 3:
## Get every user name (first and last)  
## For given song's title
 
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(song text, first_name text, last_name text, primary key (song, first_name, last_name))"
try:
    session.execute("DROP TABLE IF EXISTS songs")
    session.execute(query)
except Exception as e:
    print(e)

In [50]:
# Process csv file, fill data to cassandra
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # type convert
        line[8]=int(line[8])
        line[3]=int(line[3])
        line[5]=float(line[5])
        line[10]=int(line[10])
        try:
            # session_id int, item_in_session int, artist text, song text, song_length decimal
            query = "INSERT INTO sessions(session_id, item_in_session, artist, song, song_length)\
                    VAlUES (%s, %s, %s, %s, %s)"
            session.execute(query, (line[8], line[3], line[0], line[9], line[5]))
        except Exception as e:
            print(e)
        try:
            # user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text
            query = "INSERT INTO users (user_id, session_id, item_in_session, artist, song, first_name, last_name) \
                    VAlUES (%s, %s, %s, %s, %s, %s, %s)"
            session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)
        try:
            # song text, first_name text, last_name text
            query = "INSERT INTO songs(song, first_name, last_name) VAlUES (%s, %s, %s)"
            session.execute(query, (line[9], line[1], line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [51]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("SELECT * FROM sessions LIMIT 5")
except Exception as e:
    print(e)

print("\n\nsessions:")
for row in rows:
    print(row)
    
try:
    rows = session.execute("SELECT * FROM users LIMIT 5")
except Exception as e:
    print(e)

    
print("\n\nusers:")
for row in rows:
    print(row)
    
try:
    rows = session.execute("SELECT * FROM songs LIMIT 5")
except Exception as e:
    print(e)

print("\n\nsongs:")
for row in rows:
    print(row)



sessions:
Row(session_id=23, item_in_session=0, artist='Regina Spektor', song='The Calculation (Album Version)', song_length=Decimal('191.08526'))
Row(session_id=23, item_in_session=1, artist='Octopus Project', song='All Of The Champs That Ever Lived', song_length=Decimal('250.95791'))
Row(session_id=23, item_in_session=2, artist='Tegan And Sara', song='So Jealous', song_length=Decimal('180.06159'))
Row(session_id=23, item_in_session=3, artist='Dragonette', song='Okay Dolores', song_length=Decimal('153.39057'))
Row(session_id=23, item_in_session=4, artist='Lil Wayne / Eminem', song='Drop The World', song_length=Decimal('229.58975'))


users:
Row(user_id=58, session_id=768, item_in_session=0, artist='System of a Down', first_name='Emily', last_name='Benson', song='Sad Statue')
Row(user_id=58, session_id=768, item_in_session=1, artist='Ghostland Observatory', first_name='Emily', last_name='Benson', song='Stranger Lover')
Row(user_id=58, session_id=768, item_in_session=2, artist='Evergr

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [52]:
## Query 1:  get the artist, song title and song's length 
## for sessionId = 338, and itemInSession = 4
try:
    rows = session.execute("SELECT artist, song, song_length FROM sessions WHERE session_id = 338 and item_in_session=4")
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [53]:
## Query 2: get name of artist, song (sorted by itemInSession) and user (first and last name)\
## For userid = 10, sessionid = 182

try:
    rows = session.execute("SELECT artist, song, first_name, last_name FROM users WHERE user_id =10 and session_id= 182 ORDER BY item_in_session")
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [54]:
## Query 3: get every user name (first and last)  
## For song = 'All Hands Against His Own't

try:
    rows = session.execute("SELECT first_name, last_name FROM songs WHERE song = 'All Hands Against His Own'")
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)
                    

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [55]:
try:
    session.execute("DROP TABLE IF EXISTS sessions")
except Error as e:
    print(e)
try:
    session.execute("DROP TABLE IF EXISTS users")
except Error as e:
    print(e)
try:
    session.execute("DROP TABLE IF EXISTS songs")
except Error as e:
    print(e)

### Close the session and cluster connection¶

In [56]:
session.shutdown()
cluster.shutdown()